In [15]:
from models.vcwe import VCWE
from utils.collate import max_pad
from utils.preprocess import Preprocess
import gc
import pickle
import time
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from fastprogress import master_bar, progress_bar

In [ ]:
device = torch.device('cuda:0')
lr = 0.001
max_epoch = 1
batch_size = 256
encode_dim = 200
char_dim = 200
atten_hidden = 256
dataset_path = Path("data/dataset.pkl")
result_path = Path("result")

print('##### load dataset #####')

with dataset_path.open("rb") as rf:
    dataset = pickle.load(rf)
data_flatten = []
for d in dataset:
    data_flatten.extend(d)
word2freq = Counter(data_flatten)
unique_words = list(word2freq.keys())
unique_words.sort()
word2idx = {w: i for i, w in enumerate(unique_words)}
idx2word = {i: w for i, w in enumerate(unique_words)}
freq = np.array([word2freq[w] for w in unique_words])
n_vacab = len(unique_words)

del unique_words, word2freq, data_flatten
gc.collect()

##### load dataset #####


In [ ]:
data = Preprocess(dataset, word2idx, idx2word, freq, n_vacab)

In [ ]:
data_loader = torch.utils.data.DataLoader(data, batch_size, shuffle=True, collate_fn=max_pad, num_workers=4)

In [ ]:
model = VCWE(encode_dim, n_vacab, char_dim, atten_hidden).to(device)

In [14]:
%%time
for pos_u, pos_v, pos_img, neg_v, neg_img in progress_bar(data_loader, parent=mb):
        print('loader cpmplete')
        pos_u = pos_u.to(device)
        pos_v = pos_v.to(device)
        pos_img = pos_img.to(device)
        neg_v = neg_v.to(device)
        neg_img = neg_img.to(device)
        loss = model(pos_u, pos_v, pos_img, neg_v, neg_img)
        print(f'loss: {loss.item()} and end 1 iter')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('end optimize')

KeyboardInterrupt: 